In [ ]:
#!pip install folium

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium.plugins import MarkerCluster
from folium import Choropleth
import geopandas as gpd
import os
import warnings
warnings.filterwarnings('ignore')


In [ ]:
def fetch_files_path(dir_path):
    file_paths=[]
    files = os.listdir(path)
    csv_files = [file for file in files if file.endswith(".csv")]
    for csv_file in csv_files:
        file_paths.append(os.path.join(path, csv_file))
    return file_paths

In [ ]:
path= "C:\\Users\\LENOVO\\OneDrive - University Of Houston (1)\\Desktop\\UH\\@Projects\\Crime_Analysis_cleaned_data"
list_files=fetch_files_path(path)

In [ ]:
df_dict={}
k=2009
for i in list_files:
    df=pd.read_csv(i)
    df_dict[f'df_{k}']=df
    k+=1

In [ ]:
df_dict.keys()

In [ ]:
for i in df_dict.keys():
    print(i,df_dict[i].shape)

In [ ]:
df_dict['df_2020'].columns

In [ ]:
df_dict['df_2023'].columns

In [ ]:
col=list(df_dict['df_2015'].columns)
print("List of extra columns:")
for i in df_dict.keys():
    for i in list(df_dict[i].columns):
        if i not in col:
            print(i)
            col.append(str(i))

In [ ]:
print("List of all columns:\n",col)

### Replacing columns with calculated columns. (Only when both of them are present in the data)

In [ ]:
#replacing date with mode_date column and Beat with Calculated Beat column
#only when either of the replacement is available in the dataframe
for i in df_dict.keys():
    col=list(df_dict[i].columns)
    if ('Mod_Date' in col) and ('Date' in col):
        df_dict[i]=df_dict[i].drop(columns=['Date'])
        df_dict[i]=df_dict[i].rename(columns={'Mod_Date': 'Date'})
    if ('Calculated_Beat' in col) and ('Beat' in col):
        df_dict[i]=df_dict[i].drop(columns=['Beat'])
        df_dict[i]=df_dict[i].rename(columns={'Calculated_Beat': 'Beat'})
    if ('Street Name' in col):
        df_dict[i]=df_dict[i].rename(columns={'Street Name': 'StreetName'})
    if ('Block Range' in col):
        df_dict[i]=df_dict[i].rename(columns={'Block Range': 'BlockRange'})
    if ('NIBRS Class' in col):
        df_dict[i]=df_dict[i].rename(columns={'NIBRS Class': 'NIBRSClass'})
    if ('NIBRS Class' in col):
        df_dict[i]=df_dict[i].rename(columns={'ZIP Code': 'ZIPCode'})

In [ ]:
for i in df_dict.keys():
    print(i,df_dict[i].shape)

In [ ]:
col=list(df_dict['df_2015'].columns)
print("List of extra columns:")
for i in df_dict.keys():
    for i in list(df_dict[i].columns):
        if i not in col:
            print(i)
            col.append(str(i))

In [ ]:
col=list(df_dict['df_2015'].columns)
col

In [ ]:
k = 2019
plt.rcParams['figure.figsize'] = (20, 9)
for i in range(5):
    plt.subplot(3,2,i+1)
    plt.style.use('bmh')
    sns.countplot(df_dict[f'df_{k}']['NIBRSClass'], palette = 'gnuplot')
    plt.title(f'Major Crimes in Houston for {k}', fontweight = 30, fontsize = 20)
    plt.xticks(rotation = 90)
    k+=1
plt.tight_layout()
plt.show()

### 2022-2023 dataset contains the latitude and longitude data

In [ ]:
df_coordinates = pd.concat([df_dict['df_2023'], df_dict['df_2022']])
df_coordinates.reset_index(drop=True, inplace=True)
df_coordinates = df_coordinates.dropna(subset = ['MapLongitude', 'MapLatitude'])

In [ ]:
df_coordinates.head()

In [ ]:
df_coordinates.shape

In [ ]:
plt.rcParams['figure.figsize'] = (20, 9)
plt.style.use('bmh')
sns.countplot(df_coordinates['NIBRSClass'], palette = 'gnuplot')
plt.title('Major Crimes in Houston for 2022-2023', fontweight = 30, fontsize = 20)
plt.xticks(rotation = 90)
plt.show()

In [ ]:
map = folium.Map(location=[29.7604, -95.3698], zoom_start=10)

for index, row in df_coordinates[410000:].iterrows():
    folium.CircleMarker(location=[row['MapLatitude'], row['MapLongitude']], radius=0.5, color='red', fill_color='red', fill_opacity=1).add_to(map)

# Display the map
map

In [ ]:
# Create a Folium map centered around Houston
houston_map = folium.Map(location=[29.7604, -95.3698], zoom_start=11)

# Define the crime levels based on some criteria (you can adjust this based on your data)
low_crime_threshold = 1
high_crime_threshold = 5

# Create MarkerClusters for better visualization of multiple markers
marker_cluster_low = MarkerCluster(name='Low Crime')
marker_cluster_moderate = MarkerCluster(name='Moderate Crime')
marker_cluster_high = MarkerCluster(name='High Crime')

# Iterate over the rows of the DataFrame and add markers to the appropriate MarkerCluster
df_tmp=df_coordinates[40000:]
for index, row in df_tmp.iterrows():
    if row['Offenses'] < low_crime_threshold:
        marker_cluster_low.add_child(folium.Marker(location=[row['MapLatitude'], row['MapLongitude']],
                                                   popup=row['Offense Type'],
                                                   icon=folium.Icon(color='blue')))
    elif low_crime_threshold <= row['Offenses'] < high_crime_threshold:
        marker_cluster_moderate.add_child(folium.Marker(location=[row['MapLatitude'], row['MapLongitude']],
                                                         popup=row['Offense Type'],
                                                         icon=folium.Icon(color='yellow')))
    else:
        marker_cluster_high.add_child(folium.Marker(location=[row['MapLatitude'], row['MapLongitude']],
                                                     popup=row['Offense Type'],
                                                     icon=folium.Icon(color='red')))

# Add MarkerClusters to the map
houston_map.add_child(marker_cluster_low)
houston_map.add_child(marker_cluster_moderate)
houston_map.add_child(marker_cluster_high)

# Add LayerControl for toggling marker clusters
folium.LayerControl().add_to(houston_map)

# Save or display the map
houston_map.save('crime_map.html')  # Save to an HTML file
houston_map


In [ ]:
# Assuming df is your DataFrame with the crime data
# Assuming geojson_file is your GeoJSON file with Houston's geographical boundaries
geojson_file = gpd.read_file("C:\\Users\\LENOVO\\OneDrive - University Of Houston (1)\\Desktop\\UH\\@Projects\\Crime_analysis\\COH_ZIPCODE.geojson")
# Assuming 'ZIPCode' is the common column for merging
geojson_file.head()

In [ ]:
geojson_file.info()

In [ ]:
# Assuming 'ZIPCode' is the column you want to convert to float
geojson_file['Zip_Code'] = pd.to_numeric(geojson_file['Zip_Code'], errors='coerce').astype(float)

In [ ]:
merged_data = pd.merge(geojson_file, df_coordinates, left_on='Zip_Code', right_on='ZIPCode', how='left')
houston_map = folium.Map(location=[29.7604, -95.3698], zoom_start=11)

In [ ]:
Choropleth(
    geo_data=geojson_file,
    name='choropleth',
    data=merged_data,
    columns=['Zip_Code', 'Offenses'],
    key_on='feature.properties.Zip_Code',
    fill_color='RdYlGn',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Crime Intensity'
).add_to(houston_map)

folium.LayerControl().add_to(houston_map)

# Save or display the map
houston_map.save('choropleth_map.html')  # Save to an HTML file
houston_map

## 2019-2023 dataset 

In [ ]:
columns_to_drop = ['MapLongitude', 'MapLatitude']
df1= df_coordinates.drop(columns=columns_to_drop)

df_2019_2023 = pd.concat([df1, df_dict['df_2021'], df_dict['df_2020'], df_dict['df_2019']])
df_2019_2023.reset_index(drop=True, inplace=True)

In [ ]:
df_2019_2023.isnull().sum()

In [ ]:
df_2019_2023['NIBRSClass'] = df_2019_2023['NIBRSClass'].astype(str)
df_2019_2023['Offense Type'] = df_2019_2023['Offense Type'].astype(str)
df_2019_2023['Beat'] = df_2019_2023['Beat'].astype(str)

In [ ]:
crime_counts = df_2019_2023['Offense Type'].value_counts().sort_values(ascending=False).nlargest(20)
plt.figure(figsize=(10,4))
# create countplot with sorted values
sns.countplot(x='Offense Type', data=df_2019_2023, order=crime_counts.index, palette='plasma',orient="h")

# set axis labels and title
plt.xlabel('')
plt.ylabel('Count')
plt.title('Major Crimes in Houston for 2019 - 2023', fontweight=10, fontsize=10)

# rotate x-axis labels for readability
plt.xticks(rotation=90)

# display plot
plt.show()

In [ ]:
df_2019_2023['NIBRSClass'].value_counts().head(10)

In [ ]:
df_2019_2023['Offense Type'].unique()

In [ ]:
top_nbrs_classes = df_2019_2023['NIBRSClass'].value_counts().index[:10].tolist()

for nbrs_class in top_nbrs_classes:
    nbrs_desc = df_2019_2023.loc[df_2019_2023['NIBRSClass'] == nbrs_class, 'Offense Type'].values[0]
    print(f'{nbrs_class}: {nbrs_desc}')

In [ ]:
# create new column for month
df_2019_2023['Date']=pd.to_datetime(df_2019_2023['Date'])
df_2019_2023['Month'] = df_2019_2023['Date'].dt.month
# group by month and crime, count occurrences, and select top 5
plt.figure(figsize=(6,10))
top_crimes=df_2019_2023.groupby(['Month', 'NIBRSClass']).size().sort_values(ascending=False).groupby('Month').nlargest(3)
top_crimes = top_crimes.reset_index(level=0, drop=True).reset_index()
# create bar plot
sns.catplot(data=top_crimes, x='Month', y=0, hue='NIBRSClass', kind='bar', height=6, aspect=2, palette='Set2')
plt.yticks([0,4000,8000,12000])
plt.xticks(list(range(0,12)),['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])
# set axis labels and title
plt.xlabel('Month')
plt.ylabel('Count')
plt.title('Top 3 Crimes by Month for the years 2019-2023')

# display plot
plt.show()

In [ ]:
# Get the top 20 values for Premise and NIBRSClass
top_beat = df_2019_2023['Beat'].value_counts().head(20).index.tolist()
top_nibrsclass = df_2019_2023['NIBRSClass'].value_counts().head(20).index.tolist()

# Filter the dataframe to only include the top 10 values
df_filtered = df_2019_2023.loc[(df_2019_2023['Beat'].isin(top_beat)) & (df_2019_2023['NIBRSClass'].isin(top_nibrsclass))]

# Create a pivot table and normalize the data
beat_by_type = df_filtered.pivot_table(values='Incident', index='Beat', columns='NIBRSClass', aggfunc=np.size).fillna(0)

In [ ]:
from sklearn.cluster import AgglomerativeClustering as AC
    
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result


df_lt = normalize(beat_by_type)
ix = AC(3).fit(df_lt.T).labels_.argsort() # a trick to make better heatmaps
plt.figure(figsize=(11,5))
plt.imshow(df_lt.T.iloc[ix,:], cmap='Reds')
plt.colorbar(fraction=0.03)
plt.xticks(np.arange(df_lt.shape[0]), df_lt.index, rotation='vertical')
plt.yticks(np.arange(df_lt.shape[1]), df_lt.columns)
plt.title('Normalized Top 20 Beats frequency for top 20 crimes')
plt.grid(False)
plt.show()

### Clustering of data based on premise and type of crime 

In [ ]:
# Get the top 15 values for Premise and NIBRSClass
top_premise = df_2019_2023['Premise'].value_counts().head(15).index.tolist()
top_nibrsclass = df_2019_2023['NIBRSClass'].value_counts().head(15).index.tolist()

# Filter the dataframe to only include the top 10 values
df_filtered = df_2019_2023.loc[(df_2019_2023['Premise'].isin(top_premise)) & (df_2019_2023['NIBRSClass'].isin(top_nibrsclass))]

# Create a pivot table and normalize the data
location_by_type = df_filtered.pivot_table(values='Incident', index='Premise', columns='NIBRSClass', aggfunc=np.size).fillna(0)

In [ ]:
df_lt = normalize(location_by_type)
ix = AC(3).fit(df_lt.T).labels_.argsort() # a trick to make better heatmaps
plt.figure(figsize=(11,5))
plt.imshow(df_lt.T.iloc[ix,:], cmap='Reds')
plt.colorbar(fraction=0.03)
plt.xticks(np.arange(df_lt.shape[0]), df_lt.index, rotation='vertical')
plt.yticks(np.arange(df_lt.shape[1]), df_lt.columns)
plt.title('Normalized Top 15 locations frequency for top 15 crimes')
plt.grid(False)
plt.show()

In [ ]:
df_tmp = df_2019_2023.set_index('Date') 
plt.figure(figsize=(15,7))
df_tmp.resample('M').size().plot(legend=False)
plt.title('Number of crimes per month (2019 - 2023)')
plt.xlabel('Months')
plt.ylabel('Number of crimes')
plt.show()

In [ ]:
color = plt.cm.viridis(np.linspace(0, 1, 30))
df_tmp['Hour'].value_counts().sort_values(ascending=False).plot.bar(color = color, figsize = (10, 2))
plt.title('Distribution of crime over the day', fontsize = 20)
plt.show()

## 2009-2018 dataset 

In [ ]:
list(df_dict.keys())[:-6]

In [ ]:
list_df=[]
for i in list(df_dict.keys())[:-6]:
    list_df.append(df_dict[i])

In [ ]:
df = pd.concat(list_df)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df['Offense Type'].unique()

In [ ]:
df=df[df['Offense Type']!='1']

In [ ]:
df['Offense Type'].unique()

In [ ]:
df['Offense Type'].value_counts()

In [ ]:
df['Date']=pd.to_datetime(df['Date'])
df_1 = df.set_index('Date') 
plt.figure(figsize=(15,8))

plt.subplot(2,1,1)
df_1.resample('M').size().plot(legend=False)
plt.title('Number of crimes per month (2009 - 2017)')
plt.xlabel('Year')
plt.ylabel('Number of crimes')
plt.show()
plt.figure(figsize=(8,5))

plt.subplot(2,1,2)
sns.countplot(x='Offense Type', data=df, order=df['Offense Type'].value_counts().sort_values(ascending=False).index, palette='plasma',orient="h")
plt.title('Major Crimes in Houston for 2019 - 2023', fontweight=10, fontsize=10)
plt.yticks([0, 200000, 400000, 600000], ['0', '2L', '4L', '6L'])
plt.xlabel("")
plt.ylabel("")
plt.xticks(rotation=90)
plt.show()

In [ ]:
df['Date'] = pd.to_datetime(df['Date'])
df_1 = df.set_index('Date')

plt.figure(figsize=(15, 7))
plt.yticks([0, 2000, 4000, 6000,8000,10000,12000], ['0', '2k', '4k', '6k','8k','10k','12k'])
df_1.resample('M').size().plot(legend=False)
plt.title('Number of crimes per month (2009 - 2017)')
plt.xlabel('Year')
plt.ylabel('Number of crimes')

ax_minor = plt.axes([0.65, 0.33, 0.18, 0.2])  # [left, bottom, width, height]
plt.yticks([0, 200000, 400000, 600000], ['0', '2L', '4L', '6L'])
sns.countplot(x='Offense Type', data=df, order=df['Offense Type'].value_counts().sort_values(ascending=False).index, palette='plasma',orient="h")
plt.xticks(rotation=90)
ax_minor.set_xlabel('') 
ax_minor.set_ylabel('')
ax_minor.set_title('Major Offense Type in Houston for 2009-17', fontweight=10, fontsize=10)
ax_minor.tick_params(axis='x', rotation=90)
plt.show()

 ### Prediction using Facebook Prophet model